In [ ]:
!pip install transformers==4.40.0
!pip install -q torch torchvision pydicom opencv-python Pillow accelerate

#GPU update: pip install torch torchvision transformers==4.40.0 pydicom opencv-python Pillow accelerate


In [ ]:
# Install dependencies
#Imports
import os, glob
import torch
import pydicom
import cv2
from PIL import Image
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "StanfordAIMI/CheXagent-2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    low_cpu_mem_usage=True
)
##device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
###########seems to thin that calling the if torch.cudo.is_available() as a param should work!!
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define query
question = "Does this chest X-ray show a pneumothorax?"


In [ ]:
for name, param in model.named_parameters():
    if param.device.type == "meta":
        print(f"{name} is on the meta device")

In [ ]:
# attach the colab project to teh files
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#Set paths
from pathlib import Path
ROOT = Path('/content/drive/MyDrive/MLProjects/foundation-models-radiology')
DICOM_DIR = ROOT / 'PTXHeadtoHeadSmall'   # use the exact folder name as on Drive
print("exists:", ROOT.exists())

#ROOT = Path("/content/drive/MyDrive")  # adjust if needed
#DICOM_DIR = ROOT / "PTXHeadtoHeadSmall"
JPEG_DIR = Path("/content/drive/MyDrive/MLProjects/foundation-models-radiology/cxr_jpegs")
JPEG_DIR.mkdir(exist_ok=True)

#Replace with: direct file access or use rclone to sync your Google Drive into the pod’s local filesystem.
#RunPod doesn’t support drive.mount().
#GPU update:
#ROOT = Path('/workspace/MLProjects/foundation-models-radiology')
#JPEG_DIR = ROOT / 'cxr_jpegs'



In [ ]:
# Convert DICOMs to JPEGs
def dicom_to_jpeg(dicom_path, jpeg_path):
    ds = pydicom.dcmread(str(dicom_path))
    img = ds.pixel_array
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    img = cv2.equalizeHist(img)  # optional contrast enhancement
    cv2.imwrite(str(jpeg_path), img)

# Batch convert
#dicom_paths = list(DICOM_DIR.rglob("*.dcm"))
# find DICOMs (case-insensitive .dcm)
dicom_paths = list(DICOM_DIR.rglob('*.[dD][cC][mM]'))
print("count:", len(dicom_paths))
print("samples:", dicom_paths[:5])

jpeg_paths = []
for dcm_path in dicom_paths:
    jpg_path = JPEG_DIR / f"{dcm_path.stem}.jpg"
    dicom_to_jpeg(dcm_path, jpg_path)
    jpeg_paths.append(str(jpg_path))


In [ ]:
# Run inference on each image
def ask_chexagent(image_path, question):
    query = tokenizer.from_list_format([
        {"image": image_path},
        {"text": question}
    ])
    conversation = [
        {"from": "system", "value": "You are a helpful assistant."},
        {"from": "human", "value": query}
    ]

    inputs = tokenizer.apply_chat_template(conversation, add_generation_prompt=True, return_tensors="pt")
    #This returns a dictionary with: input_ids: tokenized input attention_mask: mask for padding
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        do_sample=False,
        num_beams=1,
        temperature=1.0,
        top_p=1.0,
        use_cache=True,
        max_new_tokens=128
    )[0]

    response = tokenizer.decode(output, skip_special_tokens=True)
    #response = tokenizer.decode(output[input_ids.size(1):-1])
    return response.strip()

In [ ]:
# Run on all images and collect responses
import csv

results = []
for path in jpeg_paths:
    answer = ask_chexagent(path, question)
    results.append((path, answer))
    print(f" {Path(path).name} → {answer}")

with open("chexagent_results.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Answer"])
    writer.writerows(results)
